In [1]:
import torch
from torch.autograd import Variable
from torchvision import models
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import dataset
import torch.optim as optim
from finetune import PrunningFineTuner
from ModifiedModels import ModifiedResNet
import time
from torchvision import models
import random
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset_dir = "Cat_Dog_data/"
train_path = dataset_dir+"train"
test_path = dataset_dir+"test"

In [4]:
train_data_loader = dataset.loader(train_path, size = 256)
test_data_loader = dataset.test_loader(test_path, size = 256)

In [5]:
resnet = ModifiedResNet()
resnet.eval()

ModifiedResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=

In [6]:
epoches = 5
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.0001, momentum=0.9)

In [7]:
def train(model, optimizer, epoches, train_data_loader,test_data_loader, device):
    
    for i in range(epoches):
        print("Epoch: ", i, end="  ")
        for i, (batch, label) in enumerate(train_data_loader):
            
            batch = batch.to(device)
            label = label.to(device)
            model.zero_grad()
            batch = Variable(batch)
            criterion(model(batch), Variable(label)).backward()
            optimizer.step()

        test(model, test_data_loader, device)

In [8]:
def test(model, test_data_loader, device):
    correct = 0
    total = 0
    for i, (batch, label) in enumerate(test_data_loader):
        batch = batch.to(device)
        output = model(Variable(batch))
        pred = output.data.max(1)[1]
        correct += pred.cpu().eq(label).sum()
        total += label.size(0)

    print("Accuracy :", float(correct) / total)

In [9]:
#Initial accuracy 
test(resnet, test_data_loader, device)

#train fine tuned model
start = time.time()
train(resnet, optimizer, epoches, train_data_loader, test_data_loader, device)
end = time.time()
print("training time: ",end - start, " secs.")

RuntimeError: DataLoader worker (pid 4259) is killed by signal: Bus error. It is possible that dataloader's workers are out of shared memory. Please try to raise your shared memory limit.

In [ ]:
torch.save(resnet, "models/resnet")